In [ ]:
from utils import get_zhipu_api_key
ZHIPU_API_KEY = get_zhipu_api_key()
llm_config = {
    'api_key': ZHIPU_API_KEY,
    "model": "glm-4",
    'base_url': "https://open.bigmodel.cn/api/paas/v4/"
    }

In [ ]:
from autogen.coding import LocalCommandLineCodeExecutor
from autogen_tools import *
executor = LocalCommandLineCodeExecutor(
    timeout=60,
    work_dir="coding",
    functions=[get_company_info, search_company_name_by_info, get_company_register, search_company_name_by_register, get_sub_company_info, search_company_name_by_sub_info, get_legal_document, search_case_num_by_legal_document]
)

In [ ]:
from autogen import ConversableAgent, AssistantAgent

code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="NEVER",
    default_auto_reply=
    "请继续 如果所有的事情都已经做完，请返回'TERMINATE'。",
)

In [ ]:
from prompt import WRITER_PROMPT
code_writer_agent = AssistantAgent(
    name="code_writer_agent",
    system_message=WRITER_PROMPT,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [ ]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message="请查询景津装备股份有限公司所属的行业类别，并告知在该行业分类下共有多少家公司？",
)
chat_result.chat_history[-3]['content']

In [ ]:
# result
from utils import *
from tqdm import tqdm
queries = read_jsonl('./question.json')
results = []
for query in tqdm(queries):
    try:
        results.append(code_executor_agent.initiate_chat(code_writer_agent, message=query['question'])[-3]['content'])
    except Exception as e:
        results.append(query['question'])